In [1]:
import pandas as pd
import requests

def get_data():
    path = '/home/nicolas/Descargas/SCRIPT.xlsx'
    df = pd.read_excel(path)
    #los nombres de las columnas estan en la fila 4
    df.columns = df.iloc[2]
    #aplicamos trim a los nombres de las columnas
    df.columns = df.columns.str.strip()
    #eliminamos las filas de la 0 a la 4
    df = df[3:]
    #eliminamos las columnas que tienen todos los valores en NaN
    df = df.dropna(axis=1, how='all')
    return df

#Index(['N', 'Pantalla', 'Funcionalidad ', 'Tipo de prueba ', 'Sistema Op',
    #    'Tipo de Usuario', 'Paso a paso', 'Testers', 'Éxito Función',
    #    'Éxito Diseño', 'Caso Exitoso'],
    #   dtype='object', name=2)

In [2]:
print(get_data().columns)

Index(['N', 'Pantalla', 'Funcionalidad', 'Tipo de prueba', 'Sistema Op',
       'Tipo de Usuario', 'Paso a paso', 'Testers', 'Éxito Función',
       'Éxito Diseño', 'Caso Exitoso'],
      dtype='object', name=2)


In [4]:
#pantalla, Funcionalidad, Tipo de prueba, Sistema Op, Tipo de Usuario se deben crear en el sistema antes de subir los casos de prueba

pantallas = []
funcionalidades = []
tipos_prueba = []
sistemas_op = []
tipos_usuario = []

def get_items_in_db():
    global pantallas
    global funcionalidades
    global tipos_prueba
    global sistemas_op
    global tipos_usuario
    
    locations = [
        {
            'lista': pantallas,
            'endpoint': 'pantalla'
        },
        {
            'lista': funcionalidades,
            'endpoint': 'funcionalidad'
        },
        {
            'lista': tipos_prueba,
            'endpoint': 'tipo_prueba'
        },
        {
            'lista': sistemas_op,
            'endpoint': 'so'
        },
        {
            'lista': tipos_usuario,
            'endpoint': 'tipo_usuario'
        }
    ]
    
    for location in locations:
        response = requests.get(f'https://udlatester-api.lerolero.com.ec/{location["endpoint"]}/',params={'pagination': {}})
        if response.status_code == 200:
            location['lista'] = response.json()['data']
        else:
            raise Exception(f'Error al obtener {location["endpoint"]}. {response.text}')
        
    

def get_or_create(tipo, nombre):
    lista = []
    if tipo == 'pantalla':
        lista = pantallas
        endpoint = 'pantalla'
    elif tipo == 'funcionalidad':
        lista = funcionalidades
        endpoint = 'funcionalidad'
    elif tipo == 'tipo_prueba':
        lista = tipos_prueba
        endpoint = 'tipo_prueba'
    elif tipo == 'sistema_op':
        lista = sistemas_op
        endpoint = 'so'
    elif tipo == 'tipo_usuario':
        lista = tipos_usuario
        endpoint = 'tipo_usuario'
    
    coincidencia = list(filter(lambda x: x['nombre'] == nombre, lista))
    if len(coincidencia) == 0:
        data = {
            'aplicacion_id': 1,
            'nombre': nombre
        }
        response = requests.post(f'https://udlatester-api.lerolero.com.ec/{endpoint}/', json=data)
        if response.status_code == 200:
            lista.append(response.json())
            return response.json()
        else:
            raise Exception(f'Error al crear {tipo} {nombre}. {response.text}')
    else:
        return coincidencia[0]
        
        
def create_test_case(row):
    pantalla = get_or_create('pantalla', row['Pantalla'])
    funcionalidad = get_or_create('funcionalidad', row['Funcionalidad'])
    tipo_prueba = get_or_create('tipo_prueba', row['Tipo de prueba'])
    sistema_op = get_or_create('sistema_op', row['Sistema Op'])
    tipo_usuario = get_or_create('tipo_usuario', row['Tipo de Usuario'])
    
    data = {
        'aplicacion_id': 1,
        'pantalla_id': pantalla['id'],
        'funcionalidad_id': funcionalidad['id'],
        'tipo_prueba_id': tipo_prueba['id'],
        'so_id': sistema_op['id'],
        'tipo_usuario_id': tipo_usuario['id'],
        'paso_a_paso': row['Paso a paso'],
    }
    
    response = requests.post('https://udlatester-api.lerolero.com.ec/caso_prueba/', json=data,  headers={
                                                                                  
        'Content-Type': 'application/json'
    })
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Error al crear caso de prueba. {response.text}')



    

In [5]:
get_items_in_db()

Exception: Error al obtener funcionalidad. <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>udlatester-api.lerolero.com.ec | 502: Bad gateway</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-li

In [59]:
for index, row in get_data().iterrows():
    create_test_case(row)
    
    